# This program loads features created in "0A AIS data download, clean & process.ipynb" to create a machine learning classifier for application to the Cuebiq data

## Key inputs: 
* **Indicators2019_All.C.csv.csv** -- a csv file labeled data with features for AIS trips
## Key output: 
* Random Forest classifier: **rf_model_AIS_2019.pkl**

# Preliminaries

In [ ]:
import pandas as pd
import os
import sys
import time
import csv
from datetime import datetime, timedelta
from datetime import datetime
import pytz  
import warnings
import numpy as np
import math
local_timezone = pytz.timezone('US/Central')

import csv
import datetime
import datetime as dt
import geopandas as gpd
import heapq
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pytz
import requests
import seaborn as sns
import time
import math

from scipy import interpolate
from shapely import wkt
from shapely.geometry import Polygon
from zipfile import ZipFile
from shapely.geometry import Point  # Import the Point class from shapely.geometry
from datetime import datetime

# from pandarallel import pandarallel
# pandarallel.initialize()

local_timezone = pytz.timezone('US/Central')

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


# Functions

In [ ]:
# THIS CODE GENERATE THE SEASON BASED ON MONTH

# def month_to_season(month_num):
    
#     monthMar= (month_num>1)*(month_num-1) + (month_num<=1)*(11+month_num)
#     season=int((monthMar) / 3)+1
    
#     return season
# month 6 and 1 are wrong

def month_to_season(month_num):
    if (month_num == 11) | (month_num == 0) | (month_num == 1): # DEC-FEB
        season = 4
    if (month_num == 2) | (month_num == 3) | (month_num == 4): # MARCH - MAY
        season = 1
    if (month_num == 5) | (month_num == 6) | (month_num == 7): # JUNE - August 
        season = 2
    if (month_num == 8) | (month_num == 9) | (month_num == 10): # SEP- NOV
        season = 3
    return season

# check season function
season= month_to_season(1)
season

# note 0 equals Jan and 11 equals december 
def monthofyear(EpochTime):  # January = 0
    Base = 1546322400  # Jan. 1, 2019, 12:00 a.m. 
    Jan12021 = 731
    Jan12022 = 1096
    Jan12023 = 1461

    FebStart = 31
    MarStart = 59
    AprStart = 90
    MayStart = 120
    JunStart = 151
    JulStart = 181
    AugStart = 212
    SepStart = 243
    OctStart = 273
    NovStart = 304
    DecStart = 334
    leapyear2020 = 1582869600     # Feb 29, 2020

    days_since_2019 = epoch_to_days_since_1_1_2019(EpochTime)
    leapyearadjust = -1*(EpochTime >leapyear2020)
    year = int((days_since_2019+ leapyearadjust)/365)
    dayofyear = (days_since_2019) - 365*year +  leapyearadjust
    month = 1
    month = 1*(dayofyear>=FebStart) + \
            1*(dayofyear>=    MarStart ) + \
            1*(dayofyear>=    AprStart ) + \
            1*(dayofyear>=    MayStart ) + \
            1*(dayofyear>=    JunStart ) + \
            1*(dayofyear>=    JulStart ) + \
            1*(dayofyear>=    AugStart ) + \
            1*(dayofyear>=    SepStart ) + \
            1*(dayofyear>=    OctStart ) + \
            1*(dayofyear>=    NovStart ) + \
            1*(dayofyear>=    DecStart )
    return month
        

def epoch_to_season(EpochTime):
    month = monthofyear(EpochTime)
    season = month_to_season(month)
    return season

def epoch_to_DOW(EpochTime):
    Base = 1546322400  # Jan. 1, 2019, 12:00 a.m. 
    BaseDOW = 1
    DaySince = (EpochTime - Base) / (24*60*60)
    WeeksSinceBase = DaySince / 7
    DayOfWeek = BaseDOW + int((WeeksSinceBase - int(WeeksSinceBase)) * 7)
    return DayOfWeek

def epoch_to_days_since_1_1_2019(EpochTime):
    Base = 1546322400  # Jan. 1, 2019, 12:00 a.m. 
    DaySinceBase = int((EpochTime-Base) / (60*60*24))
    return DaySinceBase

def AISdate_to_epoch(AISDate):
    from datetime import datetime

    # date_string = "2019-06-01T16:14:14"

    # Define the format of the input date string
    # date_format = "%Y-%m-%dT%H:%M:%S"
    date_format = "%Y-%m-%d %H:%M:%S"


    # Convert the date string to a datetime object
    dt_object = datetime.strptime(AISDate, date_format)

    # Convert the datetime object to epoch time
    epoch_time = int(dt_object.timestamp())

    # print("Epoch Time:", epoch_time)
    return epoch_time

def epoch_to_hour_of_day(EpochTime):
    Base = 1546322400  # Jan. 1, 2019, 12:00 a.m. 
    # Daylight Savings Time points for US Central Time
    start2019 = 1552204800  
    end2019 = 1572768000
    start2020 = 1583654400
    end2020 = 1604217600
    start2021 = 1615708800
    end2021 = 1636272000
    start2022 = 1647158400
    end2022 = 1667721600
    start2023 = 1678608000
    end2023 = 1699171200

    
    DayLightSavingsAdjust = +1 * (EpochTime > start2019) + \
                            -1 * (EpochTime > end2019) + \
                            +1 * (EpochTime > start2020) + \
                            -1 * (EpochTime > end2020) + \
                            +1 * (EpochTime > start2021) + \
                            -1 * (EpochTime > end2021) + \
                            +1 * (EpochTime > start2022) + \
                            -1 * (EpochTime > end2022) + \
                            +1 * (EpochTime > start2023) + \
                            -1 * (EpochTime > end2023)
#    print(DayLightSavingsAdjust)
    DaysSince = ((EpochTime-Base) / (60*60*24))
    PortionOfDay = DaysSince - int(DaysSince)
    HourOfDay = int(PortionOfDay*24) + DayLightSavingsAdjust
    return HourOfDay

def epoch_to_date(epoch_time):
    # Convert epoch time to a datetime object
    dt = datetime.fromtimestamp(epoch_time)
    
    # Format the datetime as 'YYYY-MM-DD'
    formatted_date = dt.strftime('%Y-%m-%d')
    
    return formatted_date

# Set directories to be used

In [ ]:
#################################################################
# Directory for this output
OurTable_V3_directory = '~/RecFishing/Analysis with Our Tables and V3/Data Files'
# Expand the tilde to the user's home directory
OurTable_V3_directory = os.path.expanduser(OurTable_V3_directory)
# Check to make sure the directory exist
DirExist = os.path.exists(OurTable_V3_directory)
print(OurTable_V3_directory, "exists = " ,DirExist)

#################################################################
# Directory some core data fro analysis
CoreData_Directory = '~/RecFishing/CoreData'
CoreData_Directory = os.path.expanduser(CoreData_Directory)
print(CoreData_Directory, "exists = ", os.path.exists(CoreData_Directory))

#################################################################
# Directory  with Original_directory material
Original_directory = '~/RecFishing/DataflowStudioJobs'
Original_directory = os.path.expanduser(Original_directory)
DirExist = os.path.exists(Original_directory)
print(Original_directory, "exists = ", DirExist)


#################################################################
# Directory  with Original_directory material
Previously_Processed_directory = '~/RecFishing/DataflowStudioJobs/FinalCode - Rec Fishing Identification'
Previously_Processed_directory = os.path.expanduser(Previously_Processed_directory)
DirExist = os.path.exists(Previously_Processed_directory)
print(Previously_Processed_directory, "exists = ", DirExist)

#################################################################
# Directory with Travel Cost files
Travel_Cost_directory = '~/RecFishing/Travel Costs with Dedicated Table/CSV Files'
# Expand the tilde to the user's home directory
Travel_Cost_directory = os.path.expanduser(Travel_Cost_directory)
DirExist = os.path.exists(Travel_Cost_directory)
print(Travel_Cost_directory, "exists = ", DirExist)

#################################################################
# Directory with Weather data and related files
Weather_Data_directory = '~/RecFishing/uploaded_files/Weather Data'
# Expand the tilde to the user's home directory
Weather_Data_directory = os.path.expanduser(Weather_Data_directory)
print(Weather_Data_directory, "exists = ", DirExist)

####################################################################################
####################  AIS Directory #################################################
AIS_Directory = '~/RecFishing/AIS Files/Data'
AIS_Directory = os.path.expanduser(AIS_Directory)
DirExist = os.path.exists(AIS_Directory)
print(AIS_Directory, "exists = ", DirExist)

# ID_list_RandomSample from ScheduledExecution5.pkl


# Set input and output files to be used

In [ ]:
def check_file_existence(file_path):
    if os.path.exists(file_path):
        print(f"Does     Exist {file_path}")
    else:
        print(f"Does NOT Exist {file_path}")
        
# Complete list of randomized IDs
# PKL_File_With_Random_IDs_Filename  =  os.path.join(Original_directory, 'cuebiq_id_list.pkl')
# check_file_existence(PKL_File_With_Random_IDs_Filename)

# Complete list of randomized IDs- without bernouli sampling 740k
# PKL_File_With_Random_IDs_Filename  =  os.path.join(Original_directory, 'cuebiq_id_list_wo_sampling_740k.pkl')
PKL_File_With_Random_IDs_Filename  =  os.path.join(CoreData_Directory, 'cuebiq_id_list_wo_sampling_740k.pkl')
check_file_existence(PKL_File_With_Random_IDs_Filename)
    
# Data gathered and used prior to the NOAA Webinar in February 2024
IDs_Used_in_NOAA_Webinar_filename = os.path.join(CoreData_Directory, 'IDs_From_Random_Draw_Prior_to_NOAA_Webinar.csv')
check_file_existence(IDs_Used_in_NOAA_Webinar_filename)
Ping_Used_in_NOAA_Webinar_filename = os.path.join(CoreData_Directory, 'Pings_From_Random_Draw_Prior_to_NOAA_Webinar.csv')
check_file_existence(Ping_Used_in_NOAA_Webinar_filename)
                                                     
# List of IDs that have been processed for Indicators
AlreadyFullyProcessedIDs_Filename  =  os.path.join(OurTable_V3_directory, 'RandomlyChosenCuebiq_ids.List_of_Processed_ids.csv')
check_file_existence(AlreadyFullyProcessedIDs_Filename)
    
######################################################################################################################
#########################  ID Checklist with columns for ID, Pings, Indicators Created (TF) & Trips  ################
IDs_Pulled_from_Dedicated_Table_filename  =  os.path.join(OurTable_V3_directory, 'IDs_Pulled_From_Dedicated_Table.csv')
check_file_existence(IDs_Pulled_from_Dedicated_Table_filename)
    
# Pings in the OurTable for a single large draw of IDs TEMPORARY FILE
Pings_OurTable_temp_filename = os.path.join(OurTable_V3_directory,'Pings_OurTable_temp.csv')
check_file_existence(Pings_OurTable_temp_filename)
    
# Pings from V3 corresponding with the IDs found in the OurTable for a single large draw of IDs TEMPORARY FILE
Pings_V3_temp_filename = os.path.join(OurTable_V3_directory,'Pings_V3_temp.csv')
check_file_existence(Pings_V3_temp_filename)
    
# Set output file names
Indicators_IDs_checked_filename = os.path.join(OurTable_V3_directory, 'IDs_Checked_Indicators_OurTable.csv')
check_file_existence(Indicators_IDs_checked_filename)

cuebiq_id_list_and_count_filename= os.path.join(OurTable_V3_directory,'cuebiq_id_list_and_count.csv')
check_file_existence(cuebiq_id_list_and_count_filename)


# List of IDs and dates for V3 query Pings in the OurTable for a single large draw of IDs TEMPORARY FILE
OurTable_IDs_and_Dates_filename = os.path.join(OurTable_V3_directory,'OurTable_IDs_and_Dates.csv')
check_file_existence(OurTable_IDs_and_Dates_filename)
    
# Filenames for consolidated data frames
Pings_OurTable_Gulf_filename= os.path.join(OurTable_V3_directory,'Pings_OurTable_Gulf_ALL.csv')
check_file_existence(Pings_OurTable_Gulf_filename)

Pings_OurTable_Gulf_MT19_filename= os.path.join(OurTable_V3_directory,'Pings_OurTable_Gulf_MT19.csv')
check_file_existence(Pings_OurTable_Gulf_MT19_filename)

Pings_OurTable_Coast_filename= os.path.join(OurTable_V3_directory,'Pings_OurTable_Coast.csv')
check_file_existence(Pings_OurTable_Coast_filename)

Pings_OurTable_Outside_our_wkts_filename= os.path.join(OurTable_V3_directory,'Pings_OurTable_Outside_our_wkts.csv')
check_file_existence(Pings_OurTable_Outside_our_wkts_filename)
    
##########################################################################################
############################### INDICATORS  ##############################################
Indicators_filename = os.path.join(OurTable_V3_directory,'Indicators_OurTable.csv')
check_file_existence(Indicators_filename)
     
AIS_indicators_file_path = os.path.join(AIS_Directory,'Indicators2019_All.C.csv')
check_file_existence(AIS_indicators_file_path)
  

##########################################################################################
############################### Random Forest Stuff    ###############################
classifier_file_path = os.path.join(AIS_Directory, f'rf_model_AIS_2019.pkl')

RF_Results_test = os.path.join(AIS_Directory, f'Test_observations.csv')
RF_Results_Train = os.path.join(AIS_Directory, f'Train_observations.csv')
RF_Results_filename = os.path.join(AIS_Directory, f'RandomForest_Predictions2019AISData.csv')
  
##########################################################################################
############################### Weather data files ####################################
Buoys_file_path  = os.path.join(Weather_Data_directory,'Buoys.csv')
check_file_existence(Buoys_file_path)

Weather_file_path  = os.path.join(Weather_Data_directory,'DailyWeatherData.csv')
check_file_existence(Weather_file_path)
    
### Dedicate Table Names for reference
# Dedicated table with all Pings within the Gulf WKT for 1/12019 - 4/22/2022
#  Table Name:  dedicated.ScheduledExecution5.DeviceTable   
#  Code used for call:  RecFishing/DataflowStudioJobs/ScheduledEx5-updated.ipynb

# Dedicated table with all Pings within the Gulf WKT AND Origin for 1/12019 - 4/22/2022
#  Table Name:  dedicated.ScheduledExecution5_parallel_origin.DeviceTable
#  Code used for call:  RecFishing/DataflowStudioJobs/ScheduledEx5-origin.ipynb


# Function to correct the trip duration and related variables

In [ ]:
import pandas as pd

def df_duration_correction(df):
    # Calculate corrected trip duration
    df['Trip_Duration_t_Corrected'] = (df['timestamp_end_t'] - df['timestamp_start_t']) / 60
    
    # Calculate duration correction factor and apply to other columns
    df['Duration_Correction'] = df['Trip_Duration_t'] / df['Trip_Duration_t_Corrected']
    df['Pings_per_minute_t'] = df['Pings_per_minute_t'] * df['Duration_Correction']
    df['pct_time_stopped'] = df['pct_time_stopped'] * df['Duration_Correction']
    df['pct_time_trawling'] = df['pct_time_trawling'] * df['Duration_Correction']
    df['pct_time_moving'] = df['pct_time_moving'] * df['Duration_Correction']
    
    # If Trip_Duration_t_Corrected is zero, set all related columns to zero
    zero_duration_mask = df['Trip_Duration_t_Corrected'] == 0
    df.loc[zero_duration_mask, ['Pings_per_minute_t', 'pct_time_stopped', 'pct_time_trawling', 'pct_time_moving']] = 0
    
    # Update Trip_Duration_t to corrected value
    df['Trip_Duration_t'] = df['Trip_Duration_t_Corrected']
    
    # Drop intermediate columns
    df.drop(columns=['Trip_Duration_t_Corrected', 'Duration_Correction'], inplace=True)
    
    return df

def duration_correction(timestamp_end_t, timestamp_start_t, Trip_Duration_t, 
                        Pings_per_minute_t, pct_time_stopped, pct_time_trawling, 
                        pct_time_moving):

    Trip_Duration_t_Corrected = (timestamp_end_t - timestamp_start_t) / 60
    
    print("Trip_Duration_t, Trip_Duration_t_Corrected", Trip_Duration_t, Trip_Duration_t_Corrected)
    if Trip_Duration_t_Corrected > 0:
        Duration_Correction = Trip_Duration_t / (Trip_Duration_t_Corrected)

        Pings_per_minute_t = Pings_per_minute_t * Duration_Correction
        pct_time_stopped = pct_time_stopped * Duration_Correction
        pct_time_trawling = pct_time_trawling * Duration_Correction
        pct_time_moving = pct_time_moving * Duration_Correction
    else:
        Duration_Correction = 0

        Pings_per_minute_t = 0
        pct_time_stopped = 0
        pct_time_trawling = 0
        pct_time_moving = 0

    Trip_Duration_t =Trip_Duration_t_Corrected
    return Trip_Duration_t, Pings_per_minute_t, pct_time_stopped, pct_time_trawling, pct_time_moving

# Assuming df is your DataFrame containing the columns mentioned
# df['Trip_Duration_t'], df['Pings_per_minute_t'], df['pct_time_stopped'], df['pct_time_trawling'], df['pct_time_moving'] = \
#     duration_correction(df['timestamp_end_t'], df['timestamp_start_t'], df['Trip_Duration_t'], 
#                         df['Pings_per_minute_t'], df['pct_time_stopped'], df['pct_time_trawling'], 
#                         df['pct_time_moving'])

# Random Forest Classifier

In [ ]:
import datetime
import joblib


# # Read df -- not needed if run sequentially
AISIndicators_df= pd.read_csv(AIS_indicators_file_path)
###### Correct the trip durations

AISIndicators_df['timestamp_start_t'] = AISIndicators_df['timestamp_start_t'].apply(AISdate_to_epoch)
AISIndicators_df['timestamp_end_t'] = AISIndicators_df['timestamp_end_t'].apply(AISdate_to_epoch)
AISIndicators_df = df_duration_correction(AISIndicators_df)


# Identify columns to be used in RF
X = AISIndicators_df[[
                        'trips_per_day', 
                        'break_duration_t',
                        'Weekday_trips', 
                        'Trip_Duration_t',
                        'Total_distance_traveled_t', 
                        'Distance_from_origin_t', 
                        'first_distance_from_coast_t',
                        'last_distance_from_coast_t', 
                        'Max_distance_traveled_origin_t',
                        'Weekend_trip_t', 
                        'time_stopped_t', 
                        'pct_time_stopped',
                        'number_of_stops_t', 
                        'longest_stop_t',
                        'shortest_stop_t',
                        'time_trawling_t',
                        'pct_time_trawling',
                        'number_of_trawl_t',
                        'longest_trawl_t', 
                        'shortest_trawl_t', 
                        'distance_trawling_t',
                        'time_moving_t', 
                        'pct_time_moving', 
                        'number_of_moves_t',
                        'longest_move_t', 
                        'shortest_move_t',
                        'distance_moving_t',
                        'dist_from_origin_during_stops', 
                        'Trip_pings_mov_traw_per_time_t',
                        'move_efficiency_t', 
                        'move_speed_t',
                        'maxspeed_t', 
                        'max_distance_from_coast_t',
                        'WSPD', 
                        'GST',
                        'WVHT', 
                        'ATMP'  
                        ]]

y = AISIndicators_df['vt']

identifiers = AISIndicators_df[['mmsi', 'Trip_number', 'vt']]

# Split the data into training and testing sets
#  NOTE THAT TEST_SIZE  CAN BE MODIFIED -- 20% IS, ACCORDING TO CHAT GPT, THE DEFAULT TEST_SIZE
X_train, X_test, y_train, y_test, identifiers_train, identifiers_test = train_test_split(X, y, identifiers, test_size=0.2, random_state=49)

# Initialize the Random Forest classifier
# rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Initialize the Random Forest classifier, Weight using the balance_subsample option
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42, class_weight="balanced_subsample")

# Train the classifier on the training data
rf_classifier.fit(X_train, y_train)

###############  -- RF analysis is done --###############  

# Predict vessel types on the testing data
y_pred_test = rf_classifier.predict(X_test)

# Basic evaluation of model's accuracy on the test data
accuracy = accuracy_score(y_test, y_pred_test)
print(f"Accuracy: {accuracy:.4f}")

# Predict vessel types on the testing data
y_pred_train = rf_classifier.predict(X_train)
accuracy_train = accuracy_score(y_train, y_pred_train )
print(f"Accuracy in training: {accuracy_train:.4f}")

# Assuming you have trained rf_classifier and want to save it

# Save the trained Random Forest classifier to a file
import datetime

current_datetime = datetime.datetime.now()
date_string = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")

# classifier_file_path = os.path.join(Indicatorsdir, f'rf_model_{date_string}.pkl')
joblib.dump(rf_classifier, classifier_file_path)
print("Done")

# Create a single array with all the data, the predicted class and the probabilities of each class. This uses df's from the previous cell

In [ ]:
################################################################
# CREATE A DATA FRAME FOR THE TEST OBSERVATIONS FROM ABOVE
################################################################
# Create a combined data frame for all the test data
test_df = identifiers_test
# ### print
# print(1,test_df.shape)
test_df['group'] = "test"
test_df['vt_pred'] = y_pred_test
# ### print
# print(2,test_df.shape)

test_df.reset_index(drop=True, inplace=True)
predicted_proba_df = []
# predicted_proba_df.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)

predicted_probabilities = rf_classifier.predict_proba(X_test)
predicted_proba_df = pd.DataFrame(predicted_probabilities, columns=rf_classifier.classes_)
test_df = pd.concat([test_df, predicted_proba_df], axis=1)
test_df = pd.concat([test_df, X_test], axis=1)
# ### print
# print(3,test_df.shape)
# Export test_df to a CSV file
test_df.to_csv(RF_Results_test, index=False)

################################################################
# CREATE A DATA FRAME FOR THE TRAINING OBSERVATIONS FROM ABOVE
################################################################
# Create a combined data frame for all the train data
train_df = identifiers_train
# ### print
# print(1,train_df.shape)
train_df['group'] = "train"
train_df['vt_pred'] = y_pred_train
# ### print
# print(2,train_df.shape)

train_df.reset_index(drop=True, inplace=True)
predicted_proba_df.reset_index(drop=True, inplace=True)
X_train.reset_index(drop=True, inplace=True)

predicted_probabilities = rf_classifier.predict_proba(X_train)
predicted_proba_df = pd.DataFrame(predicted_probabilities, columns=rf_classifier.classes_)
train_df = pd.concat([train_df, predicted_proba_df], axis=1)
train_df = pd.concat([train_df, X_train], axis=1)
# ### print
# print(3,train_df.shape)
# Export train_df to a CSV file
train_df.to_csv(RF_Results_Train, index=False)

################################################################
# PULL TEST AND TRAIN RESULTS TOGETHER INTO A SINGLE DATA FRAME AND EXPORT
################################################################
current_datetime = datetime.datetime.now()
date_string = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")

full_df = pd.concat([train_df, test_df], axis=0)
full_df.to_csv(RF_Results_filename, index=False)
print("Done")